In [ ]:
# PI3Kδ Ligand Preparation and Docking Pipeline
This notebook includes automated ligand conversion (PDB → PDBQT) using MGLTools and batch docking with AutoDock Vina.

In [ ]:
# Batch ligand preparation using MGLTools for AutoDock
# Converts S1–S10.pdb files to .pdbqt format

import os
import subprocess

# Fixed paths for MGLTools python and script
mgltools_python_path = r"C:\Program Files (x86)\MGLTools-1.5.6\python.exe"
prepare_ligand_script = r"C:\Program Files (x86)\MGLTools-1.5.6\Lib\site-packages\AutoDockTools\Utilities24\prepare_ligand4.py"

# Replace with your actual directories
input_folder = r"PATH_TO_YOUR_PDB_FILES"
output_folder = r"PATH_TO_SAVE_PDBQT_FILES"

# Ensure output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process all PDB files from S1.pdb to S10.pdb
for i in range(1, 11):
    pdb_file = os.path.join(input_folder, f"S{i}.pdb")
    pdbqt_file = os.path.join(output_folder, f"lig{i}.pdbqt")

    if os.path.exists(pdb_file):
        print(f"Processing {pdb_file} ...")

        command = [
            mgltools_python_path,
            prepare_ligand_script,
            "-l", pdb_file,
            "-o", pdbqt_file,
            "-A", "hydrogens"
        ]

        try:
            result = subprocess.run(command, capture_output=True, text=True)
            print(result.stdout)  # Success messages
            if result.stderr:
                print("Warnings/Errors:", result.stderr)
        except Exception as e:
            print(f"Error processing {pdb_file}: {e}")
    else:
        print(f"File {pdb_file} not found. Skipping.")

print("Batch conversion complete!")

In [ ]:
# Automated docking using AutoDock Vina
# Loops through all ligands and logs best affinity values

import subprocess
import os
import glob

# Define paths - replace with actual paths
vina_path = r"PATH_TO_VINA_EXECUTABLE"
receptor = r"PATH_TO_RECEPTOR_PDBQT"
ligand_folder = r"PATH_TO_LIGAND_PDBQT_FILES"
output_folder = r"PATH_TO_OUTPUT_FOLDER"
config = r"PATH_TO_VINA_CONFIG_FILE"

summary_file = os.path.join(output_folder, "docking_summary.csv")

# Ensure output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all ligand files in the ligand folder
ligands = glob.glob(os.path.join(ligand_folder, "*.pdbqt"))

# Check if Vina executable, receptor, and config exist
if not os.path.exists(vina_path):
    print("Error: AutoDock Vina executable not found.")
    exit(1)
if not os.path.exists(receptor):
    print("Error: Receptor file not found.")
    exit(1)
if not os.path.exists(config):
    print("Error: Configuration file not found.")
    exit(1)

# Prepare summary file with header
with open(summary_file, "w") as f:
    f.write("Ligand,Best_Affinity_kcal_per_mol\n")

# Run docking for each ligand
for ligand in ligands:
    ligand_name = os.path.basename(ligand).replace(".pdbqt", "")
    output_file = os.path.join(output_folder, f"{ligand_name}_out.pdbqt")
    log_file = os.path.join(output_folder, f"{ligand_name}_log.txt")

    vina_command = [
        vina_path,
        "--receptor", receptor,
        "--ligand", ligand,
        "--out", output_file,
        "--log", log_file,
        "--config", config
    ]

    try:
        print(f"Docking {ligand_name}...")
        subprocess.run(vina_command, check=True)
        print(f"Docking completed for {ligand_name}.")
    except subprocess.CalledProcessError:
        print(f"Error during docking for {ligand_name}. Skipping...")
        continue

    # Parse the log file for best binding affinity
    best_affinity = None
    if os.path.exists(log_file):
        with open(log_file, "r") as f:
            for line in f:
                if line.strip().startswith("-----+"):
                    next_line = next(f, None)
                    if next_line:
                        try:
                            best_affinity = float(next_line.split()[1])
                        except (IndexError, ValueError):
                            best_affinity = None
                        break

    with open(summary_file, "a") as f:
        f.write(f"{ligand_name},{best_affinity}\n")

print("Batch docking completed! Check 'docking_summary.csv' for results.")